In [ ]:
from src import *
from src.extra import ExEstimates

from src.parallelize import parallelize

In [ ]:
freq_list = np.linspace(0.1, 0.4, 61)
delay_params = (2.8e-3 + 1e-6, 0.48e-3)

In [ ]:
def ApproxCFIM(A, N=50, sigma=103):
    n = np.arange(N)
    mat_1 = [[N, 0,            0],
             [0, np.sum(n**2), np.sum(n)],
             [0, np.sum(n),    N]]
    
    mat_2 = [[1/2, 0,            0],
             [0,   2*A**2*pi**2, pi*A**2],
             [0,   pi*A**2,      A**2/2]]

    return (1/sigma**2) * np.array(mat_1) * np.array(mat_2)


def ApproxCRB(A, N=50, sigma=103):
    return np.linalg.inv(ApproxCFIM(A, N, sigma))

In [ ]:
@parallelize()
def run(f, m):
    meta = MetaData(m, np.round(f, 5), DMD.A(3))
    c = Simulator(meta, 'sign', delay_params).gen()
    # c.savez('TEMP/')
    c = c.est()
    c = ExEstimates(c).mle()
    return c

cs1, cs2 = run(freq_list, 'DI'), run(freq_list, 'SPADE')

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(2 * 8.5 / 2.54, 2 * 0.618 * 8.5 / 2.54), sharey='row', sharex=True)

for m in ('SPADE', 'DI'):
    f_var = []
    cs = cs1 if m == 'DI' else cs2
    for i, f in enumerate(freq_list):
        f_var.append(cs[i].extra_estimates_f.var(ddof=1) * cs[i].photons)

    crb = ApproxCRB((4/pi) * DMD.A(3))[1, 1]
    ax[0, 0].plot(freq_list, np.array(f_var), 'o' if m == 'SPADE' else '^', label=m)
    ax[1, 0].plot(freq_list, np.array(f_var) / crb, 'o' if m == 'SPADE' else '^', label=m)

ax[0, 0].axhline(crb, color='tab:red', zorder=1, label='QCRB')


for m in ('SPADE', 'DI'):
    f_var = []
    cs = cs1 if m == 'DI' else cs2
    for i, f in enumerate(freq_list):
        f_var.append(cs[i].estimates.var(ddof=1) * cs[i].photons)

    crb = 1 / DI.ApproxCFI((4/pi) * DMD.A(3))
    ax[0, 1].plot(freq_list, np.array(f_var), 'o' if m == 'SPADE' else '^', label=m)
    ax[1, 1].plot(freq_list, np.array(f_var) / crb, 'o' if m == 'SPADE' else '^', label=m)

ax[0, 1].axhline(crb, color='tab:red', zorder=1, label='QCRB')



for i in (0, 1):
    ax[0, i].set_xlim(0.1, 0.4)
    ax[1, i].set_xlabel(r'$f$')
    ax[0, i].legend(loc='upper left')
    ax[1, i].legend(loc='upper left')
    ax[1, i].axhline(1, color='tab:red', zorder=1)

ax[0, 0].set_ylabel(r'$\nu\cdot\,{\rm Var}\,(\hat f)$')
ax[1, 0].set_ylabel(r'$\nu\cdot\,{\rm Var}\,(\hat f)\cdot\mathcal{F}$')
ax[0, 0].set_title(r'Single Params Est; a.k.a. $\theta=f$')
ax[0, 1].set_title(r'Multi Params Est; a.k.a. $\theta=(A, f, \phi)^T$')

plt.tight_layout()
plt.show()

In [ ]:
@parallelize()
def run(f, m, method):
    meta = MetaData(m, np.round(f, 5), DMD.A(3))
    c = Simulator(meta, 'sign', delay_params).gen()
    # c.savez('TEMP/')
    # c = c.est()
    c = ExEstimates(c).mle() if method == 'mle' else ExEstimates(c).lse()
    return c

In [ ]:
cs1, cs2 = run(freq_list, 'DI', 'mle'), run(freq_list, 'SPADE', 'mle')
cs3, cs4 = run(freq_list, 'DI', 'lse'), run(freq_list, 'SPADE', 'lse')


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(2 * 8.5 / 2.54, 0.618 * 8.5 / 2.54), sharey='row', sharex=True)

for m in ('SPADE', 'DI'):
    f_var = []
    cs = cs1 if m == 'DI' else cs2
    for i, f in enumerate(freq_list):
        f_var.append(cs[i].extra_estimates_f.var(ddof=1) * cs[i].photons)

    ax[0].plot(freq_list, np.array(f_var), 'o' if m == 'SPADE' else '^', label=m)


for m in ('SPADE', 'DI'):
    f_var = []
    cs = cs3 if m == 'DI' else cs4
    for i, f in enumerate(freq_list):
        f_var.append(cs[i].extra_estimates_f.var(ddof=1) * cs[i].photons)

    ax[1].plot(freq_list, np.array(f_var), 'o' if m == 'SPADE' else '^', label=m)

crb = ApproxCRB((4/pi) * DMD.A(3))[1, 1]
for i in (0, 1):
    ax[i].set_xlim(0.1, 0.4)
    ax[i].set_xlabel(r'$f$')

    ax[i].axhline(crb, color='tab:red', zorder=1, label='QCRB')

    ax[i].legend(loc='upper left')
    ax[i].legend(loc='upper left')

ax[0].set_ylabel(r'$\nu\cdot\,{\rm Var}\,(\hat f)$')
ax[0].set_title(r'MLE')
ax[1].set_title(r'LSE')

plt.tight_layout()
plt.show()